# install underthesea, gensim

In [76]:
pip install underthesea

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
pip install gensim


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Gọi bộ dữ liệu

In [78]:
import pandas as pd

In [79]:
comments = pd.read_csv("raw_data_final.csv", encoding="utf-8-sig")

In [80]:
print(comments.shape)

comments['Emotion'].value_counts()

(11543, 2)


Emotion
Enjoyment    3777
Other        2156
Disgust      1957
Sadness      1612
Anger         909
Surprise      613
Fear          519
Name: count, dtype: int64

In [81]:
comments.describe()

,Sentence,Emotion
count,11543,11543
unique,11452,7
top,nam mô a di đà phật,Enjoyment
freq,11,3777


# Các hàm xử lý dữ liệu

In [53]:
import re

## 1. Loại bỏ null, rỗng khỏi dữ liệu

In [54]:
print("Số lượng null trong toàn bộ dữ liệu:\n", comments.isnull().sum())

Số lượng null trong toàn bộ dữ liệu:
 Sentence    0
Emotion     0
dtype: int64


In [55]:
# Loại bỏ null khỏi dữ liệu
def remove_null(df):
    df = df.dropna().reset_index(drop=True)
    return df

## 2. Chuẩn hóa về chữ thường

In [56]:
def convert_to_lowercase(text):
    return text.lower()

## 3. Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn

In [57]:
def remove_repeated_characters(text):
    """
    Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn
    Ví dụ:
    - "đẹpppp" -> "đẹp"
    - "woooow" -> "wow"
    - "yayyy" -> "yay"
    - ":))))" -> ":)"
    - ":((" -> ":("
    """
    
    # Xóa ký tự lặp lại liên tiếp, chỉ giữ lại 1 ký tự
    # Pattern: ([character])\1+ nghĩa là ký tự nào xuất hiện 2 lần trở lên liên tiếp
    # Áp dụng cho: chữ cái (a-z, A-Z), chữ tiếng Việt có dấu, số, và các ký tự đặc biệt
    
    # Xử lý tất cả các ký tự (bao gồm cả ký tự đặc biệt như :, ), (, v.v.)
    cleaned_text = re.sub(
        r'(.)\1+',  # Bắt bất kỳ ký tự nào (.) xuất hiện 2 lần trở lên liên tiếp
        r'\1',      # Thay thế bằng chỉ 1 ký tự đó
        text
    )
    
    return cleaned_text    

## 4. Thay thế các emoji và emoticon (phổ biến) thành các từ

In [58]:
def replace_emoji_to_vietnamese(text):
    # Tokenize emoticon
    emoticon_map = {
        ':)': '<cười>', '(:': '<cười>', ':-)': '<cười>', '(-:': '<cười>', '=)': '<cười>', '(=': '<cười>',
        ':(': '<buồn>', '):': '<buồn>', ':-(': '<buồn>', ')-:': '<buồn>', '=(': '<buồn>', ')=': '<buồn>',
        ':D': '<cười_lớn>', 'xD': '<cười_lớn>', 'XD': '<cười_lớn>', '=D': '<cười_lớn>', ':d': '<cười_lớn>', '=d': '<cười_lớn>',
        ':P': '<đùa>', ':p': '<đùa>', ':-P': '<đùa>', ':-p': '<đùa>',
        ';)': '<nháy_mắt>', ';-)': '<nháy_mắt>', ';D': '<nháy_mắt_cười>',
        '<3': '<tim>',
        ':/': '<hoài_nghi>', ':\\': '<hoài_nghi>', ':-/': '<hoài_nghi>', ':-\\': '<hoài_nghi>',
        ':o': '<ngạc_nhiên>', ':O': '<ngạc_nhiên>', ':-o': '<ngạc_nhiên>', ':-O': '<ngạc_nhiên>',
        '>:(': '<tức_giận>', '>:-(': '<tức_giận>', 'D:<': '<tức_giận>',
        'T_T': '<khóc>', 'T.T': '<khóc>', 'ToT': '<khóc>', ';_;': '<khóc>', 'Q_Q': '<khóc>', 'T^T': '<khóc>',
        '^_^': '<vui>', '^-^': '<vui>', '^.^': '<vui>', '^o^': '<vui>',
        '-_-': '<mệt_mỏi>', '-.-': '<chán>', '=_=': '<ngái_ngủ>',
        'o_o': '<sốc>', 'O_O': '<sốc>', 'o.o': '<sốc>', 'O.O': '<sốc>',
        '>_<': '<khó_chịu>', '>.<': '<khó_chịu>', 'x_x': '<chết>', 'X_X': '<chết>',
        ':3': '<dễ_thương>', ':v': '<đùa>', '^^': '<vui>', '>.>': '<nghi_ngờ>', '<.<': '<nghi_ngờ>',
    }
    
    # Thay thế emoticon thông thường còn lại (theo độ dài giảm dần)
    sorted_emoticons = sorted(emoticon_map.keys(), key=len, reverse=True)
    for emoticon in sorted_emoticons:
        if emoticon in text:
            token = emoticon_map[emoticon]
            # Sử dụng regex để thay thế chính xác emoticon (tránh trùng lặp với các ký tự khác)
            text = re.sub(re.escape(emoticon), f" {token} ", text)
    
    # Emoji map
    emoji_map = {
        '😀': '<cười>', '😃': '<cười_rạng_rỡ>', '😄': '<cười_tươi>', '😁': '<cười_nhe_răng>',
        '😆': '<cười_toe_toét>', '🥲': '<nụ_cười_nước_mắt>', '🤣': '<cười_lăn_lộn_trên_sàn>', '😂': '<cười_chảy_nước_mắt>',
        '😅': '<cười_ngượng_ngùng>', '🥹': '<mặt_nước_mắt_rưng_rưng>', '☺️': '<mỉm_cười>', '😊': '<cười_mỉm>',
        '🙂': '<mỉm_cười>', '🙃': '<mỉm_cười_ngược>', '😘': '<hôn>', '🥰': '<yêu_thương>',
        '😍': '<say_mê>', '😌': '<hài_lòng>', '😉': '<nháy_mắt>', '😗': '<hôn_nhẹ>', '😙': '<hôn_dễ_thương>',
        '😚': '<hôn_dịu_dàng>', '😋': '<thích_thú>', '😛': '<trêu_chọc>', '🧐': '<suy_xét>', '🤨': '<nghi_ngờ>',
        '🤪': '<điên_rồ>', '😜': '<đùa_nghịch>', '😝': '<nhăn_nhở>', '😎': '<tự_tin>', '🥳': '<ăn_mừng>',
        '😟': '<lo_lắng>', '😔': '<trầm_ngâm>', '😞': '<thất_vọng>', '😒': '<chán>', '😏': '<cười_mỉa>',
        '😕': '<bối_rối>', '🙁': '<buồn>', '☹️': '<rầu_rĩ>', '😣': '<ráng_sức>', '😖': '<bối_rối_khổ_sở>',
        '😭': '<khóc_nức_nở>', '😢': '<buồn_bã>', '🥺': '<đáng_yêu>', '😩': '<thở_dài>', '😫': '<kiệt_sức>',
        '😤': '<hậm_hực>', '😠': '<giận>', '😡': '<phẫn_nộ>', '🤬': '<chửi_thề>', '🤯': '<kinh_ngạc>', '🤩': '<ngưỡng_mộ>',
        '😳': '<ngại_ngùng>', '🥵': '<nóng_nực>', '🥶': '<lạnh_cóng>', '😱': '<hoảng_sợ>', '🤗': '<thương>',
        '😓': '<ái_ngại>', '😥': '<buồn_nhưng_nhẹ_nhõm>', '😰': '<lo_sợ>', '😨': '<sợ_hãi>', '🤔': '<suy_nghĩ>',
        '🫣': '<rụt_rè>', '🤭': '<che_miệng>', '🫢': '<ngạc_nhiên>', '🫡': '<chào_quân_sự>', '🤫': '<suỵt>',
        '🫠': '<tan_chảy>', '🤥': '<nói_dối>', '😶': '<không_nói>', '🫥': '<ẩn_mình>', '😐': '<bình_thản>',
        '🫤': '<bối_rối>', '😑': '<vô_cảm>', '😬': '<khó_xử>', '🙄': '<mỉa_mai>', '😲': '<ngạc_nhiên>',
        '😮': '<há_mồm_ngạc_nhiên>', '😧': '<bàng_hoàng>', '😦': '<bối_rối>', '😯': '<ngạc_nhiên_nhẹ>', '🥱': '<ngáp>',
        '😴': '<ngủ>', '🤤': '<thèm_thuồng>', '😪': '<buồn_ngủ>', '😮‍💨': '<thở_dài>', '🤢': '<buồn_nôn>',
        '🥴': '<chóng_mặt>', '🤐': '<ngậm_miệng>', '😵‍💫': '<hoa_mắt>', '😵': '<chóng_mặt>', '🤮': '<nôn_mửa>',
        '🤡': '<hề_hài>', '🐧': '<đùa>', '💔': '<tim_tan_nát>', '❤️‍🩹': '<tim_được_chữa_lành>', '🫂': '<ôm>',
        '👍': '<tán_thành>', '👌': '<đồng_ý>', '🙏': '<cảm_ơn>', '👏': '<vỗ_tay>',
        '💖': '<tim>', '💕': '<tim>', '💞': '<tim>', '💓': '<tim>', '💓': '<tim>', '💗': '<tim>',
        '💯': '<một_trăm_điểm>', '🔥': '<tuyệt_vời>', 
        '❤️': '<tim>', '🩷': '<tim>', '🧡': '<tim>', '💛': '<tim>', '💚': '<tim>', '💙': '<tim>', 
        '🩵': '<tim>', '💜': '<tim>', '🤎': '<tim>', '🖤': '<tim>', '🩶': '<tim>', '🤍': '<tim>',
        '🎉': '<ăn_mừng>', '😇': '<thiên_thần>', '😈': '<ác_quỷ>',
        '🫶': '<tim_bằng_tay>', '🙌': '<vui_mừng>', '🫰': '<tim_bằng_tay>',
        '✌️': '<chiến_thắng>', '🤟': '<tình_yêu_rock_and_roll>', '👋': '<vẫy_tay_chào>'
    }

    # Sắp xếp emoji theo độ dài giảm dần để đảm bảo các emoji nhiều ký tự được thay thế trước
    sorted_emojis = sorted(emoji_map.keys(), key=len, reverse=True)
    # Kiểu có mấy cái emoji ❤️\u200d🩹 = ❤️‍🩹

    for emoji in sorted_emojis:
        if emoji in text:
            token = emoji_map[emoji]
            # Thay thế tất cả các lần xuất hiện của emoji bằng token và thêm khoảng trắng xung quanh
            text = text.replace(emoji, f"{token}")

    # Loại bỏ các token emoji lặp lại liên tiếp (ví dụ: <tim> <tim> -> <tim>)
    for token in set(emoji_map.values()):
        text = text.replace(f"{token} {token}", token)
    
    return text

## 5. Chuẩn hóa từ viết tắt / từ lóng (phổ biến)

In [59]:
def normalize_short_forms(text):
    short_forms_map = {
        'đm': '<địt_mẹ>', 'dm': '<địt_mẹ>', 'vcl': '<vãi_cả_lồn>', 'clgt': '<cái_lồn_gì_thế>',
        'lol': '<cười_lớn>', 'xl': '<xin_lỗi>', 'ko': '<không>', 'k': '<không>', 'r': '<rồi>',
        'bn': '<bạn>', 'vs': '<với>', 'bik': '<biết>', 'nx': '<nữa>', 'qá': '<quá>', 'cj': '<chị>',
        'ad': '<quản_trị_viên>', 'acc': '<tài_khoản>', 'cmt': '<bình_luận>', 'clm': '<cái_lồn_má>',
        'rep': '<trả_lời>', 'ib': '<nhắn_tin_riêng>', 'cmnl': '<con_mẹ_nó_luôn>', 'vc': '<vãi_chưởng>',
        'cc': '<con_cặc>', 'ditme': '<địt_mẹ>', 'vl': '<vãi_lồn>', 'm': '<mày>', 'mk': '<mình>', 'vloz': '<vãi_lồn>',
        'mn': '<mọi_người>', 'nc': '<nói_chuyện>', 'ng': '<người>', 'nh': '<nhắn>', 'n9': '<nam_chính>', 'nu9': '<nữ_chính>',
        'thik': '<thích>', 'uk': '<ừ>', 'j': '<gì>', 'zậy': '<vậy>',
        'nhe': '<nhé>', 'cug': '<cũng>', 'dc': '<được>', 'đc': '<được>', 'in4': '<thông_tin>',
        'thg': '<thằng>', 'ngta': '<người_ta>', 'khum': '<không>', 'khôm': '<không>',
        'lun': '<luôn>', 'thui': '<thôi>', 'thoy': '<thôi>', 'iu': '<yêu>',
        'gato': '<ghen_ăn_tức_ở>', 'đx': '<được>', 'bthg': '<bình_thường>',
        'bt': '<biết>', 'hk': '<không>', 'qtrg': '<quan_trọng>', 'lm': '<lắm>',
        'p/s': '<tái_bút>', 'ibox': '<nhắn_tin_riêng>', 'bjo': '<bao_giờ>', 'thks': '<cảm_ơn>',
        'tk': '<thằng>', 'đr': '<đúng_rồi>', 'cmm': '<con_mẹ_mày>', 'đmm': '<địt_mẹ_mày>',
        'vcđ': '<vãi_cả_đái>', 'vch': '<vãi_cả_chưởng>', 'ck': '<chồng>', 'vk': '<vợ>', 'ny': '<người_yêu>', 'nyc': '<người_yêu_cũ>',
        'nt': '<nhắn_tin>', 'loz': '<lồn>', 'dkm': '<địt_con_mẹ>',
        'dkmm': '<địt_con_mẹ_mày>', 'moasss': '<má>'
    }

    # Sắp xếp các từ viết tắt theo độ dài giảm dần để tránh việc thay thế một phần của từ viết tắt khác
    sorted_short_forms = sorted(short_forms_map.keys(), key=len, reverse=True)
    for short_form in sorted_short_forms:
        if short_form in text:
            full_form = short_forms_map[short_form]
            # Sử dụng regex để thay thế chính xác từ viết tắt (tránh trùng lặp với các từ khác)
            text = re.sub(r'\b' + re.escape(short_form) + r'\b', full_form, text)

    return text

## 6. Xóa các dòng có dấu thanh bị tách rời

In [ ]:
â --> a^

In [60]:
# Ví dụ 1 dòng có dấu thanh bị tách rời
comments['Sentence'][14]

'tui thi ́ ch va ̉ i lă ́ mày ma ̀ ăn nhỉ ̀ u no ́ người lă ́ mày mọi người anh ̣ , bi ̣ lơ ̉ miê ̣ người'

In [61]:
def remove_rows_with_separated_tone_marks(df, column_name='Sentence'):
    """
    Xóa các dòng có dấu thanh bị tách rời khỏi dữ liệu
    """
    
    # Dấu thanh Unicode combining (U+0300 - U+036F)
    tone_marks = r'[\u0300-\u036F]'  # Bao gồm: ́ ̀ ̉ ̃ ̣
    
    # Tạo mask để lọc các dòng KHÔNG chứa dấu thanh tách rời
    # Áp dụng hàm kiểm tra một chuỗi có dấu thanh tách rời không
    mask = df[column_name].apply(lambda text: not bool(re.search(tone_marks, str(text))))
    
    # Lọc DataFrame
    df_cleaned = df[mask].copy().reset_index(drop=True)
    
    return df_cleaned

In [62]:
# Tìm các index của dòng bị lỗi
def find_rows_with_tone_mark_errors(df, column_name='Sentence'):
    tone_marks = r'[\u0300-\u036F]'
    
    error_indices = []
    for idx, text in df[column_name].items():
        if re.search(tone_marks, str(text)):
            error_indices.append(idx)
    
    return error_indices

In [63]:
print("Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu:", len(find_rows_with_tone_mark_errors(comments)))

print("\nCác dòng có dấu thanh bị tách rời trong bộ dữ liệu:")
print(find_rows_with_tone_mark_errors(comments))

Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu: 48

Các dòng có dấu thanh bị tách rời trong bộ dữ liệu:
[14, 172, 318, 350, 471, 489, 512, 557, 831, 832, 988, 1062, 1099, 1588, 1937, 1968, 2073, 2178, 2433, 2464, 2682, 2929, 2942, 3170, 3224, 3579, 3713, 4239, 4436, 4517, 4842, 4847, 4985, 5207, 5290, 5294, 5539, 5567, 5611, 5847, 5926, 5974, 6109, 6453, 6488, 6544, 6677, 8441]


## 7. Xóa các ký tự đặc biệt 
*(trừ các dấu _ . : ; , ! ?)*

In [64]:
# Hàm để loại bỏ các ký tự KHÔNG PHẢI chữ cái tiếng Việt và Latin, số, '_', '.', ',', ';', '!', '?', ':'
def remove_non_alphanumeric(text):
    if pd.isnull(text) or text == "":
        return text
    # Giữ lại những ký tự mong muốn 
    cleaned_text = re.sub(r'[^0-9_.:;,!?a-zđáàảạãăắằẳặẵâấầẩậẫéèẻẹẽêềếểệễíìỉịĩóòỏọõôốồổộỗơờớởợỡúùủụũưứừửựữýỳỷỵỹ]', ' ', text)

    return cleaned_text.strip()

## 8. Chuẩn hóa chính tả
Sử dụng hàm `text_normalize` trong `underthesea`
- Chuẩn hóa Unicode
- Sửa vị trí của dấu câu: 'hoà' -> 'hòa'
- Loại bỏ khoảng trắng thừa

In [65]:
from underthesea import text_normalize

def normalize_text(text):
    # Chuẩn hóa Unicode và sửa vị trí dấu câu, loại bỏ khoảng trắng thừa
    text = text_normalize(text)
    
    return text

## 9. Kiểm tra và loại bỏ trùng lặp

### Kiểm tra

In [66]:
print("Số lượng dòng trùng lặp trong bộ dữ liệu:", len(comments[comments.duplicated(keep=False)]))

# In thử giá trị trùng lặp
comments[comments.duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong bộ dữ liệu: 130


,Sentence,Emotion
181,đọc bình luận cười ẻ,Enjoyment
848,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
943,sợ thế,Fear
1030,bài này hay quá,Surprise
1128,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1215,đáng yêu quá,Enjoyment
1275,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1936,nguy hiểm lắm,Fear
4102,dễ thương quá,Enjoyment
4386,nguy hiểm lắm,Fear


In [67]:
print("Số lượng dòng trùng lặp trong cột Sentence:", len(comments[comments['Sentence'].duplicated(keep=False)]))

# In thử giá trị trùng lặp
comments[comments['Sentence'].duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong cột Sentence: 143


,Sentence,Emotion
181,đọc bình luận cười ẻ,Enjoyment
848,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
943,sợ thế,Fear
1030,bài này hay quá,Surprise
1128,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1205,ghê vậy,Disgust
1215,đáng yêu quá,Enjoyment
1275,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1936,nguy hiểm lắm,Fear
3739,hú hồn,Fear


### Loại bỏ trùng lặp, xét trên cột Sentence

In [68]:
def remove_duplicates_sentence(df):
    df = df.drop_duplicates(subset='Sentence').reset_index(drop=True)
    return df

## 10. Tokenize

In [69]:
# Tokenize: từ các bình luận, chuyển thành các token
from underthesea import word_tokenize

def tokenize_comments(df):
    df['Tokenized_Sentence'] = df['Sentence'].apply(word_tokenize)
    return df

## 11. Loại bỏ các dấu _ . : ; ,

In [70]:
# Loại bỏ các dấu _ . : ; , sau khi tokenize
def remove_specific_punctuations_after_tokenization(tokens):
    punctuations_to_remove = {'_', '.', ':', ';', ','}
    cleaned_tokens = [token for token in tokens if token not in punctuations_to_remove]
    return cleaned_tokens

## 12. Loại bỏ các dòng rỗng ""

In [71]:
def remove_nan(df):
    df = df[df['Sentence'].str.strip() != ""].reset_index(drop=True)
    return df

## 13. Loại bỏ từ dừng

In [ ]:
vietnamese_stopwords = set([
    'và', 'là', 'có', 'của', 'cho', 'với', 'rằng', 'thì', 'lại',
    'này', 'đã', 'đang', 'sẽ', 'được', 'bị', 'ở', 'trong', 'ra', 
    'khi', 'một', 'các', 'những', 'nên', 'vì', 'như', 'thôi', 'nữa',
    'đến', 'đi', 'rất', 'quá', 'thật', 'đó', 'này', 'kia', 'vậy', 'rồi'
])

def delete_stopwords(tokens): 
    return [word for word in tokens if word not in vietnamese_stopwords]

# Pipeline xử lý dữ liệu

In [ ]:
def pipeline_preprocessing(df):
    # 1. Loại bỏ null khỏi dữ liệu
    df = remove_null(df)

    # 2. Chuyển tất cả văn bản thành chữ thường
    df['Sentence'] = df['Sentence'].apply(convert_to_lowercase)
    
    # 3. Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn
    df['Sentence'] = df['Sentence'].apply(remove_repeated_characters)
    
    # 4. Thay thế các emoji và emoticon thành các từ
    df['Sentence'] = df['Sentence'].apply(replace_emoji_to_vietnamese)
    
    # 5. Chuẩn hóa từ viết tắt, từ lóng
    df['Sentence'] = df['Sentence'].apply(normalize_short_forms)

    # 6. Xóa các dòng có dấu thanh bị tách rời khỏi dữ liệu
    df = remove_rows_with_separated_tone_marks(df)

    # 7. Xóa các ký tự đặc biệt (trừ các dấu _.;,!?)
    df['Sentence'] = df['Sentence'].apply(remove_non_alphanumeric)

    # 8. Chuẩn hóa chính tả
    df['Sentence'] = df['Sentence'].apply(normalize_text)

    # 9. Loại bỏ trùng lặp, xét trên cột Sentence
    df = remove_duplicates_sentence(df)

    # 10. Tokenize: từ các bình luận, chuyển thành các token
    df = tokenize_comments(df)

    # 11. Loại bỏ các dấu _ . : ; , sau khi tokenize
    df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(remove_specific_punctuations_after_tokenization)

    # 12. Loại bỏ các dòng rỗng ""
    df = remove_nan(df)
    
    # 13. Loại bỏ từ dừng 
    df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(delete_stopwords)


    return df

In [83]:
processed_comments = pipeline_preprocessing(comments)

In [84]:
processed_comments.describe()

,Sentence,Emotion,Tokenized_Sentence
count,11383,11383,11383
unique,11383,7,11381
top,cho mình xin bài nhạc tên là gì với ạ,Enjoyment,"[dẫu, hiên ngang, giữa, đất trời, gặp, con, vợ..."
freq,1,3695,2


In [85]:
print("Số lượng dòng bị loại bỏ sau khi xử lý:", comments.shape[0] - processed_comments.shape[0])
print("Kích thước bộ dữ liệu sau khi xử lý:", processed_comments.shape)

Số lượng dòng bị loại bỏ sau khi xử lý: 160
Kích thước bộ dữ liệu sau khi xử lý: (11383, 3)


In [86]:
print("Số lượng null trong bộ dữ liệu sau khi xử lý:")
processed_comments.isnull().sum()

Số lượng null trong bộ dữ liệu sau khi xử lý:


Sentence              0
Emotion               0
Tokenized_Sentence    0
dtype: int64

In [87]:
processed_comments['Emotion'].value_counts()

Emotion
Enjoyment    3695
Other        2147
Disgust      1937
Sadness      1588
Anger         904
Surprise      598
Fear          514
Name: count, dtype: int64

In [88]:
processed_comments.head(5)

,Sentence,Emotion,Tokenized_Sentence
0,cho mình xin bài nhạc tên là gì với ạ,Other,"[cho, mình, xin, bài, nhạc, tên, là, gì, với, ạ]"
1,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,Disgust,"[cho, đáng, đời, con, quỷ, về, nhà, lôi, con, ..."
2,ước gì sau này về già vẫn có thể như cụ này cười,Enjoyment,"[ước, gì, sau này, về, già, vẫn, có thể, như, ..."
3,mỗi lần có video của con là cứ coi đi coi lại ...,Enjoyment,"[mỗi, lần, có, video, của, con, là, cứ, coi, đ..."
4,thằng kia sao mày bắt vợ với bồ tao dọn thế ki...,Anger,"[thằng, kia, sao, mày, bắt, vợ, với, bồ tao, d..."


In [102]:
path = "cleaned_final"

processed_comments.to_csv(f"{path}\\processed_comments.csv",  index=False, encoding="utf-8-sig")

# Chia Train_Validation + Test

In [89]:
# Chia Train_Validation + Test
from sklearn.model_selection import train_test_split

train_valid_processed_comments, test_processed_comments = train_test_split(
    processed_comments, 
    test_size=0.2, 
    random_state=11, 
    stratify=processed_comments['Emotion']
    )

In [90]:
train_valid_processed_comments.shape, test_processed_comments.shape

((9106, 3), (2277, 3))

In [93]:
train_valid_processed_comments.describe()

,Sentence,Emotion,Tokenized_Sentence
count,9106,9106,9106
unique,9106,7,9106
top,sự thất bại của giáo dục,Enjoyment,"[sự, thất bại, của, giáo dục]"
freq,1,2956,1


In [94]:
test_processed_comments.describe()

,Sentence,Emotion,Tokenized_Sentence
count,2277,2277,2277
unique,2277,7,2277
top,tao lại lấy mỡ đè chết mày bây giờ,Enjoyment,"[tao, lại, lấy, mỡ, đè, chết, mày, bây giờ]"
freq,1,739,1


## Lưu file train + valid & test lại để sử dụng

In [95]:
path = "cleaned_final"

train_valid_processed_comments.to_csv(f"{path}\\train_valid_processed_comments.csv",  index=False, encoding="utf-8-sig")
test_processed_comments.to_csv(f"{path}\\test_processed_comments.csv",  index=False, encoding="utf-8-sig")